In [1]:
# --- Cellule 1 : Dépendances (à exécuter une seule fois) ---
# !pip -q install "pymupdf>=1.24.9" "transformers>=4.43" "sentencepiece>=0.1.99" "accelerate>=0.33" bitsandbytes sacremoses -U

In [2]:
SOURCE_PDF = "/Users/gauthier/Desktop/IATA_Training_Airline_Management_Career_Guide.pdf"

In [3]:
import torch

# Choix auto du device
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

In [4]:
from transformers import MarianTokenizer, MarianMTModel
MODEL_NAME = "Helsinki-NLP/opus-mt-en-fr"
MAX_NEW_TOKENS = 512                           # limite sortie par ligne
BATCH_SIZE = 8                                # taille lot traduction


tokenizer  = MarianTokenizer.from_pretrained(MODEL_NAME)
model  = MarianMTModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()

In [5]:
import fitz  # PyMuPDF

doc = fitz.open(SOURCE_PDF)

# Extraction des blocs de texte
pages_text = []
for page in doc:
    blocks = page.get_text("blocks")  # [(x0,y0,x1,y1,"texte",bloc_num,...)]
    page_blocks = []
    for b in blocks:
        text = b[4].strip()
        if len(text) >= 4:  # on ignore les bouts trop courts
            page_blocks.append((b[:4], text))  # coordonnées + texte
    pages_text.append(page_blocks)

print("Exemple de blocs :", pages_text[0][:3])

Exemple de blocs : [((102.63780212402344, 98.28779602050781, 450.6692199707031, 267.58184814453125), 'How to Make a \nCareer in Airline \nManagement:'), ((102.63780212402344, 270.8783874511719, 299.1927795410156, 308.7073974609375), 'A Comprehensive Guide')]


In [6]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

# Détecter nb de cœurs dispo et en garder 1 pour le système
MAX_WORKERS = max(1, os.cpu_count() - 1)
print(f"➡️ Utilisation de {MAX_WORKERS} threads sur {os.cpu_count()} cœurs")

def translate_batch(batch):
    """Traduit un batch de textes (utilise le modèle global, partagé en RAM)."""
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    translated = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# ---- 1. Aplatir toutes les pages ----
flat_texts = []
for page_idx, page_blocks in enumerate(pages_text, start=0):  # start=0 pour index direct
    for coords, text in page_blocks:
        flat_texts.append((page_idx, coords, text))

print(f"➡️ Total de {len(flat_texts)} blocs de texte à traduire (toutes pages confondues)")

# ---- 2. Créer des batches globaux ----
batches = [(i, [t[2] for t in flat_texts[i:i+BATCH_SIZE]]) for i in range(0, len(flat_texts), BATCH_SIZE)]

# ---- 3. Traduction en parallèle ----
all_translations = [None] * len(flat_texts)

n_batch = 0

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_idx = {executor.submit(translate_batch, batch): idx for idx, batch in batches}
    
    for future in as_completed(future_to_idx):
        idx = future_to_idx[future]
        batch_translations = future.result()
        all_translations[idx:idx+len(batch_translations)] = batch_translations
        n_batch += 1
        print(f"  - Batch {n_batch}/{len(batches)} terminé ({round(n_batch/len(batches)*100, 2)}%)")

# ---- 4. Reconstruction des pages ----
translated_pages = [[] for _ in range(len(pages_text))]

for (page_idx, coords, _), trans in zip(flat_texts, all_translations):
    translated_pages[page_idx].append((coords, trans))

print(f"\n✅ Traduction terminée : {len(translated_pages)} pages traitées")
print("\nExemple traduction première page :", translated_pages[0][:3])

➡️ Utilisation de 9 threads sur 10 cœurs
➡️ Total de 1023 blocs de texte à traduire (toutes pages confondues)
  - Batch 1/128 terminé (0.78%)
  - Batch 2/128 terminé (1.56%)
  - Batch 3/128 terminé (2.34%)
  - Batch 4/128 terminé (3.12%)
  - Batch 5/128 terminé (3.91%)
  - Batch 6/128 terminé (4.69%)
  - Batch 7/128 terminé (5.47%)
  - Batch 8/128 terminé (6.25%)
  - Batch 9/128 terminé (7.03%)
  - Batch 10/128 terminé (7.81%)
  - Batch 11/128 terminé (8.59%)
  - Batch 12/128 terminé (9.38%)
  - Batch 13/128 terminé (10.16%)
  - Batch 14/128 terminé (10.94%)
  - Batch 15/128 terminé (11.72%)
  - Batch 16/128 terminé (12.5%)
  - Batch 17/128 terminé (13.28%)
  - Batch 18/128 terminé (14.06%)
  - Batch 19/128 terminé (14.84%)
  - Batch 20/128 terminé (15.62%)
  - Batch 21/128 terminé (16.41%)
  - Batch 22/128 terminé (17.19%)
  - Batch 23/128 terminé (17.97%)
  - Batch 24/128 terminé (18.75%)
  - Batch 25/128 terminé (19.53%)
  - Batch 26/128 terminé (20.31%)
  - Batch 27/128 terminé (21

In [7]:
sizes = (
    list(range(100, 49, -10)) +
    list(range(50, 19, -5)) +
    list(range(20, 13, -1)) +
    [round(x, 1) for x in [14 - i*0.1 for i in range(int((14-4)/0.1)+1)]]
)

def best_font_size(page, rect, text, sizes):
    """Trouve la plus grande taille de police qui rentre dans rect (binary search)."""
    lo, hi = 0, len(sizes) - 1
    best = sizes[-1]

    while lo <= hi:
        mid = (lo + hi) // 2
        font_size = sizes[mid]
        rc = page.insert_textbox(
            rect,
            text,
            fontsize=font_size,
            color=(0, 0, 0),
            align=3,
            render_mode=3  # mesure uniquement
        )
        if rc >= 0:  # le texte tient
            best = font_size
            hi = mid - 1
        else:  # trop grand → réduire
            lo = mid + 1
    return best

# ---- Application séquentielle ----
for page_index, page in enumerate(doc):
    print(f"📄 Traitement page {page_index+1}/{len(doc)}")
    blocks = translated_pages[page_index]

    for coords, text in blocks:
        rect = fitz.Rect(coords)
        font_size = best_font_size(page, rect, text, sizes)

        # dessiner rectangle jaune
        page.draw_rect(rect, color=(1, 1, 0),
                       fill=(1, 1, 0),
                       fill_opacity=0.9,
                       overlay=True)

        # insérer texte à la bonne taille
        page.insert_textbox(
            rect,
            text,
            fontsize=font_size,
            color=(0, 0, 0),
            align=3
        )

OUTPUT_PDF = f"{SOURCE_PDF}_FR.pdf"
doc.save(OUTPUT_PDF)
print("✅ PDF traduit sauvegardé sous", OUTPUT_PDF)

📄 Traitement page 1/30
📄 Traitement page 2/30
📄 Traitement page 3/30
📄 Traitement page 4/30
📄 Traitement page 5/30
📄 Traitement page 6/30
📄 Traitement page 7/30
📄 Traitement page 8/30
📄 Traitement page 9/30
📄 Traitement page 10/30
📄 Traitement page 11/30
📄 Traitement page 12/30
📄 Traitement page 13/30
📄 Traitement page 14/30
📄 Traitement page 15/30
📄 Traitement page 16/30
📄 Traitement page 17/30
📄 Traitement page 18/30
📄 Traitement page 19/30
📄 Traitement page 20/30
📄 Traitement page 21/30
📄 Traitement page 22/30
📄 Traitement page 23/30
📄 Traitement page 24/30
📄 Traitement page 25/30
📄 Traitement page 26/30
📄 Traitement page 27/30
📄 Traitement page 28/30
📄 Traitement page 29/30
📄 Traitement page 30/30
✅ PDF traduit sauvegardé sous /Users/gauthier/Desktop/IATA_Training_Airline_Management_Career_Guide.pdf_FR.pdf
